In [1]:
!apt update
!apt install -y sox
!apt install -y nodejs
!apt install -y npm

!python3 -m pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!python3 -m pip install --upgrade pip wandb pandas numpy sklearn transformers datasets ipywidgets matplotlib jiwer seaborn unidecode librosa soundfile tqdm torch_audiomentations

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached wandb-0.12.11-py2.py3-none-any.whl (1.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 26.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 27.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 KB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 22.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!jupyter nbextension enable --py widgetsnbextension
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Building jupyterlab assets (build:prod:minimize)


In [3]:
!export HF_DATASETS_CACHE="/workspace/persistent/ASR/cache"
!export TRANSFORMERS_CACHE="/workspace/persistent/ASR/cache"

import os
import tempfile

cache_dir = "/workspace/persistent/ASR/cache"
temp_dir = "/workspace/persistent/ASR/temp"

os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_DATASETS_CACHE'] = cache_dir
os.environ['HF_HOME'] = cache_dir
os.environ['TMPDIR'] = temp_dir
os.environ['TEMP'] = temp_dir
os.environ['TMP'] = temp_dir

print(tempfile.gettempdir()) # prints the current temporary directory

/workspace/persistent/ASR/temp


In [4]:
from datasets import load_dataset, load_metric, ClassLabel, concatenate_datasets, Dataset, load_from_disk
import torch
import gc
import random
import pandas as pd
from IPython.display import display, HTML
import IPython.display as ipd
import numpy as np
import random
import json
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer, Wav2Vec2Processor
import torchaudio
import librosa

gc.collect()
torch.cuda.empty_cache()

In [5]:
interview_data = load_from_disk("./interview_data")

In [6]:
def convert_json(batch):
    audio = json.loads(batch['audio'])
    audio['array'] = np.array(audio['array'])
    audio['array'] = audio['array'].astype(np.float32)
    audio['sampling_rate'] = audio['sample_rate']
    batch['audio'] = audio
    return batch

interview_data = interview_data.map(convert_json)

  0%|          | 0/4134 [00:00<?, ?ex/s]

In [7]:
interview_data.features

{'path': Value(dtype='string', id=None),
 'audio': {'array': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
  'path': Value(dtype='string', id=None),
  'sample_rate': Value(dtype='int64', id=None),
  'sampling_rate': Value(dtype='int64', id=None)},
 'sentence': Value(dtype='string', id=None)}

In [8]:
common_voice_train = load_dataset("common_voice", "nl", split="train+validation", cache_dir=cache_dir)
common_voice_test = load_dataset("common_voice", "nl", split="test", cache_dir=cache_dir)

Reusing dataset common_voice (/workspace/persistent/ASR/cache/common_voice/nl/6.1.0/b879a355caa529b11f2249400b61cadd0d9433f334d5c60f8c7216ccedfecfe1)
Reusing dataset common_voice (/workspace/persistent/ASR/cache/common_voice/nl/6.1.0/b879a355caa529b11f2249400b61cadd0d9433f334d5c60f8c7216ccedfecfe1)


In [9]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [10]:
common_voice_train.features

{'path': Value(dtype='string', id=None),
 'audio': Audio(sampling_rate=48000, mono=True, _storage_dtype='struct', id=None),
 'sentence': Value(dtype='string', id=None)}

In [11]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(common_voice_train)

,path,audio,sentence
0,common_voice_nl_22916813.mp3,"{'path': 'cv-corpus-6.1-2020-12-11/nl/clips/common_voice_nl_22916813.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",Vooral vrouwen zijn het slachtoffer van de onmenselijke rechtspleging.
1,common_voice_nl_23935473.mp3,"{'path': 'cv-corpus-6.1-2020-12-11/nl/clips/common_voice_nl_23935473.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",Het zijn vrienden en iedereen is zich daarvan bewust.
2,common_voice_nl_24018053.mp3,"{'path': 'cv-corpus-6.1-2020-12-11/nl/clips/common_voice_nl_24018053.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",We heten hen van harte welkom.
3,common_voice_nl_22148903.mp3,"{'path': 'cv-corpus-6.1-2020-12-11/nl/clips/common_voice_nl_22148903.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",Dat is arrogant imperialisme.
4,common_voice_nl_23969225.mp3,"{'path': 'cv-corpus-6.1-2020-12-11/nl/clips/common_voice_nl_23969225.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",Daar is vooral op de delegaties kennis van zaken voor nodig.
5,common_voice_nl_23407328.mp3,"{'path': 'cv-corpus-6.1-2020-12-11/nl/clips/common_voice_nl_23407328.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [12]:
import re
import unidecode

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�=&\(\)]'

def remove_special_characters(batch):
    batch["sentence"] = unidecode.unidecode(batch["sentence"])
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).upper() + " "
    return batch

interview_data = interview_data.map(remove_special_characters)
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

  0%|          | 0/4134 [00:00<?, ?ex/s]

  0%|          | 0/14398 [00:00<?, ?ex/s]

Loading cached processed dataset at /workspace/persistent/ASR/cache/common_voice/nl/6.1.0/b879a355caa529b11f2249400b61cadd0d9433f334d5c60f8c7216ccedfecfe1/cache-d2bba1c3571087bd.arrow


In [13]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("facebook/hubert-xlarge-ls960-ft")

In [14]:
# interview_data.features['audio'] = common_voice_train.features['audio']

# display(type(common_voice_train[0]['audio']['array']))
# display(type(interview_data[0]['audio']['array']))

In [15]:
import torchaudio

def speech_file_to_array_fn(batch):
    batch["speech"] = batch['audio']['array']
    batch["sampling_rate"] = batch['audio']['sampling_rate']
    batch["target_text"] = batch["sentence"]
    return batch

interview_data = interview_data.map(speech_file_to_array_fn, remove_columns=interview_data.column_names)
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)

  0%|          | 0/4134 [00:00<?, ?ex/s]

  0%|          | 0/14398 [00:00<?, ?ex/s]

Loading cached processed dataset at /workspace/persistent/ASR/cache/common_voice/nl/6.1.0/b879a355caa529b11f2249400b61cadd0d9433f334d5c60f8c7216ccedfecfe1/cache-e268716e3e022a0a.arrow


In [16]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), batch["sampling_rate"], 16_000)
    batch["sampling_rate"] = 16_000
    return batch

interview_data = interview_data.map(resample, num_proc=4)
common_voice_train = common_voice_train.map(resample, num_proc=4)
common_voice_test = common_voice_test.map(resample, num_proc=4)

Loading cached processed dataset at /workspace/persistent/ASR/cache/common_voice/nl/6.1.0/b879a355caa529b11f2249400b61cadd0d9433f334d5c60f8c7216ccedfecfe1/cache-801b467cdac8c53d.arrow
Loading cached processed dataset at /workspace/persistent/ASR/cache/common_voice/nl/6.1.0/b879a355caa529b11f2249400b61cadd0d9433f334d5c60f8c7216ccedfecfe1/cache-d8fac7ff9f91a851.arrow
Loading cached processed dataset at /workspace/persistent/ASR/cache/common_voice/nl/6.1.0/b879a355caa529b11f2249400b61cadd0d9433f334d5c60f8c7216ccedfecfe1/cache-d8ac258ac455a4f8.arrow
Loading cached processed dataset at /workspace/persistent/ASR/cache/common_voice/nl/6.1.0/b879a355caa529b11f2249400b61cadd0d9433f334d5c60f8c7216ccedfecfe1/cache-e69322b9537d4079.arrow


In [17]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(interview_data))

display(interview_data[rand_int]['target_text'])
ipd.Audio(data=np.asarray(interview_data[rand_int]["speech"]), autoplay=True, rate=16000)

'DIE MOETEN ZOVEEL DINGEN MEER DOEN WAT EEN MEDEWERKER GEWOON '

In [18]:
display(common_voice_train.features)
display(interview_data.features)

common_voice_train = concatenate_datasets([common_voice_train, interview_data])

{'speech': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'sampling_rate': Value(dtype='int64', id=None),
 'target_text': Value(dtype='string', id=None)}

{'speech': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'sampling_rate': Value(dtype='int64', id=None),
 'target_text': Value(dtype='string', id=None)}

In [19]:
rand_int = random.randint(0, len(common_voice_train))

print("Target text:", common_voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train[rand_int]["sampling_rate"])

Target text: NU IS HET TIJD OM IN ACTIE TE KOMEN 
Input array shape: (49920,)
Sampling rate: 16000


In [20]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [21]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [22]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8, num_proc=4, batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.8/dist-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.8/dist-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.8/dist-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) 

In [23]:
%load_ext autoreload
%autoreload 2

from data_collator import DataCollatorCTCWithPadding

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [24]:
wer_metric = load_metric("wer")

In [25]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [26]:
from transformers import HubertForCTC

model = HubertForCTC.from_pretrained(
    "./hubert-nl-mcv-large-aug/checkpoint-166980",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [27]:
model.freeze_feature_extractor()

In [28]:
from transformers import TrainingArguments
import wandb

wandb.login()

wandb.init(project="hubert-mcv-large-aug", entity="coen22")

training_args = TrainingArguments(
  output_dir="./hubert-nl-mcv-large-aug",
  group_by_length=True,
  per_device_train_batch_size=1,
  gradient_accumulation_steps=2,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  num_train_epochs=20,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-5,
  warmup_steps=500,
  save_total_limit=2,
  report_to="wandb"
)

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)


In [29]:
from transformers import Trainer

torch.cuda.empty_cache()

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Using amp fp16 backend


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 18532
  Num Epochs = 20
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 2
  Total optimization steps = 185320
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Wer
1,0.402800,0.372069,0.204890
2,0.270600,0.377959,0.211095


***** Running Evaluation *****
  Num examples = 5708
  Batch size = 8
Saving model checkpoint to ./hubert-nl-mcv-large-aug/checkpoint-9266
Configuration saved in ./hubert-nl-mcv-large-aug/checkpoint-9266/config.json
Model weights saved in ./hubert-nl-mcv-large-aug/checkpoint-9266/pytorch_model.bin
Configuration saved in ./hubert-nl-mcv-large-aug/checkpoint-9266/preprocessor_config.json
Deleting older checkpoint [hubert-nl-mcv-large-aug/checkpoint-166980] due to args.save_total_limit
Deleting older checkpoint [hubert-nl-mcv-large-aug/checkpoint-27798] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 5708
  Batch size = 8
Saving model checkpoint to ./hubert-nl-mcv-large-aug/checkpoint-18532
Configuration saved in ./hubert-nl-mcv-large-aug/checkpoint-18532/config.json
Model weights saved in ./hubert-nl-mcv-large-aug/checkpoint-18532/pytorch_model.bin
Configuration saved in ./hubert-nl-mcv-large-aug/checkpoint-18532/preprocessor_config.json
Deleting older checkp

In [ ]:
wandb.finish()